In [1]:
from __future__ import print_function
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
cifar10_1 = unpickle("cifar-10-batches-py/data_batch_1")
cifar10_2 = unpickle("cifar-10-batches-py/data_batch_2")
cifar10_3 = unpickle("cifar-10-batches-py/data_batch_3")
cifar10_4 = unpickle("cifar-10-batches-py/data_batch_4")
cifar10_5 = unpickle("cifar-10-batches-py/data_batch_5")
cifar10_test = unpickle("cifar-10-batches-py/test_batch")

In [4]:
cifar10_rgb = np.concatenate((cifar10_1[b'data'],cifar10_2[b'data'],cifar10_3[b'data'],cifar10_4[b'data'],cifar10_5[b'data']))
cifar10_labels = np.concatenate((cifar10_1[b'labels'],cifar10_2[b'labels'],cifar10_3[b'labels'],cifar10_4[b'labels'],cifar10_5[b'labels']))

In [5]:
#cifar10cifar10_1[b'data'].shape
#cifar10_test

cifar10_test_rgb = cifar10_test[b'data']
cirfar10_test_labels = cifar10_test[b'labels']

In [6]:
# setup base epochs and batach_size

batch_size = 128
num_classes = 10
epochs = 8

In [7]:
x_train, y_train, x_test, y_test = cifar10_rgb, cifar10_labels, cifar10_test_rgb, cirfar10_test_labels

In [8]:
x_train = x_train.reshape(50000, 32 * 32 * 3)
x_test = x_test.reshape(10000, 32 * 32 * 3)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [9]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [10]:
model = Sequential()
act = keras.layers.ELU(alpha=1.0)
model.add(Dense(256, activation=act, input_shape=(32 * 32 * 3,)))
model.add(Dropout(0.2))
model.add(Dense(256, activation=act))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

/Users/xiangyuchen/anaconda3/lib/python3.6/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as ELU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [13]:
model.summary()
from keras import metrics
import keras.backend as K
def iouMetric(y_true, y_pred):
    pred = K.cast(K.greater(y_pred, 0.5), K.floatx())
    union = K.cast(K.greater(y_true + pred, 0), K.floatx())
    intersec = y_true * pred
    iou = K.sum(intersec) / (K.sum(union) + K.epsilon())

    return iou

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy', iouMetric])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test IOU:', score[2])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               786688    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 855,050
Trainable params: 855,050
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/8
50000/50000 [==============================] - 10s 198us/step - loss: 1.4751